In [ ]:
import glob
import pandas as pd
import numpy as np
from pathlib import Path
import re
import os
import gc

# Import functions provided by Competition

In [ ]:
#!git clone https://github.com/location-competition/indoor-location-competition-20.git

In [ ]:
#%cd /kaggle/working/indoor-location-competition-20/

In [ ]:
#from io_f import read_data_file

In [ ]:
inpath = '/kaggle/input/indoor-location-navigation/'
metapath = inpath + 'metadata/'
trainpath = inpath + 'train/'
testpath = inpath + 'test/'

In [ ]:
# train_buildings = os.listdir('/kaggle/input/indoor-location-navigation/train')

In [ ]:
# os.system(f'grep SiteID {testpath}/* > test_buildings.txt' )
# test_buildings = pd.read_csv('test_buildings.txt',sep='\t',header=None,names=['file','building','site'])
# test_buildings['building'] = test_buildings['building'].apply(lambda x: x[7:])
# buildings = np.unique(test_buildings['building'])

In [ ]:
# floors = []
# for b in buil:
#     floors.extend(os.listdir(os.path.join('/kaggle/input/indoor-location-navigation/train', b)))

In [ ]:
# np.unique(floors)

In [ ]:
floors = {'1F':0, '2F':1, '3F':2, '4F':3, '5F':4, '6F':5, '7F':6, '8F':7, '9F':8, 
          'B':-1, 'B1':-1,'B2':-2, 'B3':-3, 'BF':-1, 'BM':-1, 
          'F1':0, 'F2':1, 'F3':2, 'F4':3, 'F5':4, 'F6':5,'F7':6, 'F8':7, 'F9':8, 'F10':9,
          'G':0, 'LG1':-1, 'LG2':-2, 
          "L1":0,"L2":1,"L3":2,"L4":3,"L5":4,"L6":5,"L7":6,"L8":7,"L9":8,"L10":9,'L11':10, 
          'LM':0, 'M':0, 'P1':-1, 'P2':-2}


In [ ]:
os.makedirs('./cleaned/')
# os.makedirs('./cleaned/imu/')
# os.makedirs('./cleaned/waypoint/')
# os.makedirs('./cleaned/beacon/')
# os.makedirs('./cleaned/wifi/')

In [ ]:
wifi_test_dfs = pd.DataFrame()
imu_test_dfs = pd.DataFrame()
beacon_test_dfs = pd.DataFrame()

for filename in os.listdir(testpath):
        print('----'+filename)
        names = ['Time', 'Type'] + ['col'+str(x) for x in range(1,9)]
        df = pd.read_csv(testpath+filename, sep='\t', comment='#', header=None, names=names, dtype=object)
        df['path_id'] = filename.split('/')[-1][:-4]
        df['building_id'] = open(testpath+filename).readlines()[1].split('\t')[1][7:]        
        
#         l = []
#         f = pd.read_csv(testpath+filename, sep='\t', comment='#', header=None, names=names, dtype=object)
#         f['path_id'] = filename.split('/')[-1][:-4]
#         f['building_id'] = open(testpath+filename).readlines()[1].split('\t')[1][7:]
#         l.append(f)
#         df = pd.concat(l, axis=0)
    
        # wifi
        wifi_df = df[df['Type'].isin(['TYPE_WIFI'])][
            ['building_id','path_id','Time','Type','col1','col2','col3','col4','col5']].rename(
            columns={'col1':'ssid','col2':'bssid','col3':'RSSI','col4':'frequency','col5':'lastSeenTimestamp'})
        wifi_test_dfs = wifi_test_dfs.append(wifi_df)
        
        # beacon
        beacon_df = df[df['Type'].isin(['TYPE_BEACON'])][
            ['building_id','path_id','Time','Type','col1','col2','col3','col4','col5','col6','col7','col8']].rename(
            columns={'col1':'UUID','col2':'MajorID','col3':'MinorID',
                     'col4':'TxPower','col5':'RSSI','col6':'Distance','col7':'MACAddress','col8':'UnixTime'})
        beacon_test_dfs = beacon_test_dfs.append(beacon_df)
        
        # imu
        imu_df = df[df['Type'].isin(['TYPE_ACCELEROMETER','TYPE_GYROSCOPE','TYPE_MAGNETIC_FIELD','TYPE_ROTATION_VECTOR'])][
            ['building_id','path_id','Time','Type','col1','col2','col3','col4']].rename(
            columns={'col1':'x','col2':'y','col3':'z','col4':'accuracy'})
        imu_test_dfs = imu_test_dfs.append(imu_df)
        
        del df, wifi_df, beacon_df, imu_df
        gc.collect()

In [ ]:
wifi_test_dfs.to_csv(f'./cleaned/wifi_test.csv')
beacon_test_dfs.to_csv(f'./cleaned/beacon_test.csv')
imu_test_dfs.to_csv(f'./cleaned/imu_test.csv')

In [ ]:
buildings = wifi_test_dfs['building_id'].unique().tolist()#np.unique(train_buildings)
print('# of buildings:',len(buildings))

In [ ]:
del wifi_test_dfs, beacon_test_dfs, imu_test_dfs
gc.collect()

In [ ]:
wypt_dfs = pd.DataFrame()
wifi_dfs = pd.DataFrame()
imu_dfs = pd.DataFrame()
beacon_dfs = pd.DataFrame()

for building in buildings:
    print('Building ID:',building, buildings.index(building))
    
    for floor in os.listdir(trainpath+building):
        print('----',floor)
        
        names = ['Time', 'Type'] + ['col'+str(x) for x in range(1,9)]
        l = []
        for filename in glob.glob(f'{trainpath}/{building}/{floor}/*.txt'):
            f = pd.read_csv(filename, sep='\t', comment='#', header=None, names=names, dtype=object)
            f['path_id'] = filename.split('/')[-1][:-4]
            f['floor'] = floors[str(floor)]
            f['building_id'] = str(building)
            l.append(f)
        df = pd.concat(l, axis=0)
    
        # waypoint
        wypt_df = df[df['Type']=='TYPE_WAYPOINT'][
            ['building_id','floor','path_id','Time','Type','col1','col2']].rename(columns={'col1':'x','col2':'y'})
        wypt_dfs = wypt_dfs.append(wypt_df)
        
        # wifi
        wifi_df = df[df['Type'].isin(['TYPE_WIFI'])][
            ['building_id','floor','path_id','Time','Type','col1','col2','col3','col4','col5']].rename(
            columns={'col1':'ssid','col2':'bssid','col3':'RSSI','col4':'frequency','col5':'lastSeenTimestamp'})
        wifi_dfs = wifi_dfs.append(wifi_df)
        
        # beacon
        beacon_df = df[df['Type'].isin(['TYPE_BEACON'])][
            ['building_id','floor','path_id','Time','Type','col1','col2','col3','col4','col5','col6','col7','col8']].rename(
            columns={'col1':'UUID','col2':'MajorID','col3':'MinorID',
                     'col4':'TxPower','col5':'RSSI','col6':'Distance','col7':'MACAddress','col8':'UnixTime'})
        beacon_dfs = beacon_dfs.append(beacon_df)
        
#         # imu
#         imu_df = df[df['Type'].isin(['TYPE_ACCELEROMETER','TYPE_GYROSCOPE','TYPE_MAGNETIC_FIELD','TYPE_ROTATION_VECTOR'])][
#             ['building_id','floor','path_id','Time','Type','col1','col2','col3','col4']].rename(
#             columns={'col1':'x','col2':'y','col3':'z','col4':'accuracy'})
#         imu_dfs = imu_dfs.append(imu_df)

        del df, wypt_df, wifi_df, beacon_df#, imu_df
        gc.collect()

In [ ]:
wypt_dfs.to_csv(f'./cleaned/waypoint_train.csv')
wifi_dfs.to_csv(f'./cleaned/wifi_train.csv')
beacon_dfs.to_csv(f'./cleaned/beacon_train.csv')
# imu_dfs.to_csv(f'./cleaned/imu_train.csv')

In [ ]:
# del wypt_dfs, wifi_dfs, beacon_dfs, imu_dfs
# gc.collect()